In [4]:
import os

def update_file_paths(file_path):
    updated_lines = []
    
    # Dosyayı satır satır okuyun
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # Eğer satır "File path:" ile başlıyorsa
            if line.startswith("[File path:"):
                # Yolun yalnızca dosya adını almak için dosya adını ayıklıyoruz
                full_path = line.split("[File path: ")[1].strip()
                file_name = os.path.basename(full_path)
                
                # Yeni satırı "File name:" ve yalnızca dosya adı olacak şekilde oluşturuyoruz
                updated_line = f"[File name: {file_name}\n"
                updated_lines.append(updated_line)
            else:
                # Diğer satırları olduğu gibi ekliyoruz
                updated_lines.append(line)
    
    # Dosyayı güncellenmiş içerik ile tekrar yazın
    with open(file_path, 'w', encoding='utf-8') as file:
        file.writelines(updated_lines)

# Güncellemek istediğiniz dosyanın yolunu buraya yazın
file_path = '/Users/taha/Desktop/rag/test_data_routing/Mobilfunk/_summary.txt'
update_file_paths(file_path)